In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#  !pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
# !pip3 install torchvision
!pip install easydict

In [3]:
#  Created by Soniya  on 31/10/18.
#
import torch
import numpy
import scipy
import scipy.io as sio
import torch.utils.data as data_utils
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import argparse
import os
import shutil
import easydict


###------------------------------------------------------
torch.manual_seed(2019)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(2019)
###------------------------------------------------------


# D = 10;      D_min = 4; 
# D_fc = 5;    D_fc_min = 1; 
# N_fc = 20;   N_fc_min = 1;              # number of neurons in each D_fc layers

# f1 = 10; f2 = 10; f3 = 10; f4 = 10; f5 = 10; f6 = 10; f7 = 10; f8 = 10; f9 = 10; f10 = 10; 
# f11 = 10; f12 = 10; f13 = 10; f14 = 10; f15 = 10; f16 = 10; f17 = 10; f18 = 10; f19 = 10; f20 = 10; f21 = 2;

# f1 = 20; f2 = 20; f3 = 20; f4 = 20; f5 = 20; f6 = 20; f7 = 20; f8 = 20; f9 = 20; f10 = 20; 
# f11 = 20; f12 = 20; f13 = 20; f14 = 20; f15 = 20; f16 = 20; f17 = 20; f18 = 20; f19 = 20; f20 = 20; f21 = 2;

# f1 = 30; f2 = 30; f3 = 30; f4 = 30; f5 = 30; f6 = 30; f7 = 30; f8 = 30; f9 = 30; f10 = 30; 
# f11 = 30; f12 = 30; f13 = 30; f14 = 30; f15 = 30; f16 = 30; f17 = 30; f18 = 30; f19 = 30; f20 = 30; f21 = 2;

# f1 = 50; f2 = 50; f3 = 50; f4 = 50; f5 = 50; f6 = 50; f7 = 50; f8 = 50; f9 = 50; f10 = 50; 
# f11 = 50; f12 = 50; f13 = 50; f14 = 50; f15 = 50; f16 = 30; f17 = 30; f18 = 30; f19 = 30; f20 = 30; f21 = 2;

f1 = 200; f2 = 200; f3 = 200; f4 = 200; f5 = 200; f6 = 200; f7 = 200; f8 = 200; f9 = 200; f10 = 200; 
f11 = 200; f12 = 200; f13 = 200; f14 = 200; f15 = 200; f16 = 200; f17 = 30; f18 = 30; f19 = 30; f20 = 30; f21 = 2;

# f1 = 50

# C_fmin = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]; 
# C_fmax = [f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15];#,f16,f17,f18,f19,f20]


In [4]:
args = easydict.EasyDict({
    "alpha": 0.01,
    
    "phi1": 0.01,
    "phi2": 0.01,
    "phi3": 0.01,
    "phi4": 0.01,
    
    "D": 15,
    "D_min": 4,
    "D_fc": 5,          # fc layers
    "D_fc_min": 1,
    "N_fc": 128,        # neurons in fc
    "N_fc_min": 1,
    
    "C_fmin": [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    "C_fmax": [f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15],

    "num_classes": 2,
    "pop_size": 5,
    
    "epochs": 50, #,
    "batch_size": 100,
    "pop_size": 5,
    "start_epoch":0,
    "lr": 0.0001,
    "momentum": 0.9,
    "weight-decay": 1e-4,
    "print-freq": 500,
    "droprate":0.2,
    "reduce":0.5,
    
    #"resume": '/content/drive/My Drive/Normal_Op_DNet/F1_T1/best_cGA_checkpoint.pth.tar2',
    #"cGA_resume": True,
    "resume": False,
    "cGA_resume": False,
    "bottleneck": True,
    "fixed_flag": True,
    "name": '/content/drive/My Drive/Normal_Op_DNet/min1/F5_T2/',

})

args1 = easydict.EasyDict({
})


args.total_layers = args.D
args.total_filters = sum(args.C_fmax[0:args.D])

# args.f_fc = [20, 20, 20, 20, 20]#, 2]               # maximum number of neurons in each fc layer
# args.f_fc = [10, 10, 10, 10, 10]#, 2]
args.f_fc = [128, 128, 128, 128, 128]#, 2]
args.N_fc_min = [1, 1, 1, 1, 1]                      # minimum number of neurons in each fc layer
args.total_neurons = sum(args.f_fc[0:args.D_fc])

In [5]:
class Net(nn.Module):
    def __init__(self,order,F):
        super(Net, self).__init__()
        num_conv = 0; num_pool = 0; j = 0#num_relu = 0; num_BN = 0;
        self.CC1 = nn.Sequential()
        for i in range(len(order)):
            if i == 0:
                self.CC1.add_module('conv'+str(num_conv), nn.Conv2d(3, F[j], kernel_size=3, stride=2, padding=2))
                self.CC1.add_module('bn'+str(num_conv), nn.BatchNorm2d(F[j])) 
                self.CC1.add_module('relu'+str(num_conv), nn.ReLU(True))
                num_conv = num_conv + 1
                
            else:
                if order[i] == 1:
                    self.CC1.add_module('conv'+str(num_conv), nn.Conv2d(F[j], F[j+1], kernel_size=3, stride=2, padding=2))
                    self.CC1.add_module('bn'+str(num_conv), nn.BatchNorm2d(F[j+1])) 
                    self.CC1.add_module('relu'+str(num_conv), nn.ReLU(True))
                    num_conv = num_conv + 1; j = j+1
#             print(num_conv)

                if order[i] == 0:
                    self.CC1.add_module('pool'+str(num_pool), nn.MaxPool2d(kernel_size=2, stride=1))
                    num_pool = num_pool + 1
#             if order[i] == 2:
#                 self.CC1.add_module('relu'+str(num_relu), nn.ReLU(True))
#                 num_relu = num_relu + 1
#             if order[i] == 3:
#                 self.CC1.add_module('BatchNorm'+str(num_BN), nn.BatchNorm2d(F[i]))
#                 num_BN = num_BN + 1

    def forward(self, x1):
        #         print(z[a1-1])
        y11 = self.CC1(x1)
#         print('raw_cnn_out',y11.size())
        y21 = y11.view(y11.size(0),-1)
#         print('y21 in raw_cnn',y21.size())
#         y31 = self.fc(y21)
        #         print(y2)
        return y11 #, y2, y1

# raw_cnn = Net(order,F)

In [6]:
class DefineNet(nn.Module):
    def __init__(self,order,F,fc_size,neurons):
        super(DefineNet, self).__init__()
        num_conv = 0; num_pool = 0; j = 0; #num_relu = 0; num_BN = 0;
        self.CC1 = nn.Sequential()
        for i in range(len(order)):
            if i == 0:
                self.CC1.add_module('conv'+str(num_conv), nn.Conv2d(3, F[j], kernel_size=3, stride=2, padding=2))
                self.CC1.add_module('bn'+str(num_conv), nn.BatchNorm2d(F[j])) 
                self.CC1.add_module('relu'+str(num_conv), nn.ReLU())
                num_conv = num_conv + 1
            else: 
                if order[i] == 1:
                    self.CC1.add_module('conv'+str(num_conv), nn.Conv2d(F[j], F[j+1], kernel_size=3, stride=2, padding=2))
                    self.CC1.add_module('bn'+str(num_conv), nn.BatchNorm2d(F[j+1])) 
                    self.CC1.add_module('relu'+str(num_conv), nn.ReLU())
                    j = j + 1; num_conv = num_conv + 1

                if order[i] == 0:
                    self.CC1.add_module('pool'+str(num_pool), nn.MaxPool2d(kernel_size=2, stride=1))
                    num_pool = num_pool + 1
         
        self.classifier = nn.Sequential()
        for k in range(len(neurons)):
                if k == 0:
                    self.classifier.add_module('fc'+str(k), nn.Linear(fc_size*fc_size*F[j], neurons[k]))
                    self.classifier.add_module('relu'+str(k), nn.ReLU())
                    self.classifier.add_module('DR'+str(k), nn.Dropout(p=0.2,inplace=True))
                else:          # last layer in classifier 
                    self.classifier.add_module('fc'+str(k), nn.Linear(neurons[k-1], neurons[k]))
                    self.classifier.add_module('relu'+str(k), nn.ReLU())
                    self.classifier.add_module('DR'+str(k), nn.Dropout(p=0.2,inplace=True))
#                 else:
#                     self.classifier.add_module('fc'+str(k), nn.Linear(neurons[k-1], neurons[k]))
#                     self.classifier.add_module('relu'+str(k), nn.ReLU(True))

        self.fc = nn.Linear(neurons[k],2)
        self.fc1 = nn.ReLU()
        self.fc2 = torch.nn.Softmax(dim=1)
    
    
    def forward(self, x1):
            y1 = self.CC1(x1)
            # print('y1',y1.size())
            y2 = y1.view(y1.size(0),-1)
            # print('y2',y2.size())
            out = self.classifier(y2)
            # print('out1',out.size())
            out = self.fc(out)
            # print('out2', out.size())
            return out #, y2, y1

# raw_cnn = Net(order,F)

In [7]:
args.order = [1, 1, 0, 1, 0, 1, 0, 1, 0, 1] ; 
args.filters =  [101, 101, 90, 93, 93, 95]
args.fc_num = 1 
args.neurons = [55]


In [8]:
    raw_cnn = Net(args.order, args.filters)
#   print('raw_cnn', raw_cnn)
    xx = torch.rand(1, 3, 100, 100);   # 100x100 : size of cell image in the Dataset
    xx  = Variable(xx)
    out = raw_cnn(xx)
    print('out',out.size())
    fc_size = out.size(2)
    print('fc_size', fc_size)
    
    ###-----------------------------------------------------------
    ### Initialize network
    model = DefineNet(args.order, args.filters, fc_size, args.neurons)
    print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

out torch.Size([1, 95, 3, 3])
fc_size 3
Number of model parameters: 457943


In [9]:
args = easydict.EasyDict({
     "resume": '/content/drive/My Drive/cloned-repo/Public/itr_cGA_checkpoint.pth.tar60'
})

In [10]:
if args.resume:
#         if os.path.isfile(args.resume):
#             print("=> loading checkpoint '{}'".format(args.resume))
#             torch.load with map_location=torch.device('cpu')
            checkpoint = torch.load(args.resume, 'cpu')


            # epoch = checkpoint['epoch']

            tr_fit = checkpoint['CP_trfit']
            val_fit = checkpoint['CP_valfit']
            tr_losses = checkpoint['CP_trloss']
            val_losses = checkpoint['CP_valloss']
            # init_stride = checkpoint['CP_init_stride']
            # CM = checkpoint['CP_CM']
            model.load_state_dict(checkpoint['state_dict'])


In [11]:
#mat_contents =  sio.loadmat('/home/baadalvm/soniya/COIL100Data/C100Train_Data.mat')
mat_contents =  sio.loadmat('/content/drive/My Drive/Malaria/PW_Fold3/fold3_tedata.mat')
mat_data= mat_contents['data_te']
pydata = mat_data.transpose(3,2,0,1)
Test_Data = torch.FloatTensor(pydata)

#mat_labels =  sio.loadmat('/home/baadalvm/soniya/COIL100Data/C100Train_labels.mat')
mat_labels =   sio.loadmat('/content/drive/My Drive/Malaria/PW_Fold3/fold3_tedata.mat')
labels= mat_labels['labels_te']
pylabels = labels.transpose(1,0)
c = pylabels.astype(int)
py1 = torch.LongTensor(c)
Test_labels = py1.view(py1.size(0))

# train_dataset = torch.utils.data.TensorDataset(Train_Data, Train_labels)
test_dataset = torch.utils.data.TensorDataset(Test_Data, Test_labels)

# --- load datasets
# train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=args.batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

In [12]:
idx = 365, # 1101, 453, 878

input_img = Test_Data[idx]
# print('Test_labels', Test_labels[idx])
# print(input_img.size())

img = input_img.unsqueeze(0)
# print(img.size())
out_idx = model(img).argmax(dim=1)
# out_idx = output.argmax(dim=1)
# print('output', output)
# print('out_idx', out_idx)


if Test_labels[idx] == 0:
  if out_idx == 0:
    print('Correct Prediction ==> target class: parasite cell,  and predicted class: parasite cell')
  else:
    print('Incorrect Prediction ==> target class: parasite cell,  and predicted class: normal cell')
else:
  if out_idx == 0:
    print('Incorrect Prediction ==> target class: normal cell,  and predicted class: parasite cell')
  else:
    print('Correct Prediction ==> target class: normal cell,  and predicted class: normal cell')

Correct Prediction ==> target class: parasite cell,  and predicted class: parasite cell
